# Summarise MOTBX resources

Convert MOTBX resources (YAML files) to CSV file.

In [1]:
import os
import yaml
from pathlib import Path
from motbxtools import motbxschema
import pprint
pp = pprint.PrettyPrinter(indent = 2)

CWD = Path.cwd()
if CWD.name != "notebooks":
    print("Make sure to run this notebook from the 'notebooks' directory.")

MOTBX_DIR = CWD.parent # home directory of this repository

with open(MOTBX_DIR.joinpath("resources/MOTBX_version.yaml"), "r") as f:
    MOTBX_VERSION = yaml.safe_load(f)

# path to directory where resources YAML file are saved
RESOURCES_DIR = MOTBX_DIR.joinpath("resources/curated")
SUMMARY_DIR = MOTBX_DIR.joinpath("resources/summary")
if not os.path.exists(SUMMARY_DIR):
    os.mkdir(SUMMARY_DIR)
SUMMARY_CSV_LATEST = SUMMARY_DIR.joinpath(
    f"MOTBX_{MOTBX_VERSION['latest']}.csv")
SUMMARY_CSV_PREVIOUS = SUMMARY_DIR.joinpath(
    f"MOTBX_{MOTBX_VERSION['previous'][0]}.csv")
CHANGELOG_CSV = SUMMARY_DIR.joinpath(
    f"changelog_{MOTBX_VERSION['latest']}.csv")

# path to JSON SCHEMA file defining structure of MOTBX resources
SCHEMA_JSON = MOTBX_DIR.joinpath("schema/motbxschema.json")

TEST_RESOURCES_DIR = MOTBX_DIR.joinpath("tests/resources_pass")
TEST_SUMMARY_CSV = MOTBX_DIR.joinpath("tests/resources.csv")

Create YAML file summary for test resources.

In [2]:
test_collection = motbxschema.MotbxCollection(TEST_RESOURCES_DIR, SCHEMA_JSON)
test_collection.summarise(TEST_SUMMARY_CSV, validate=True)

Create summary for curated resources

In [3]:
motbx_collection = motbxschema.MotbxCollection(RESOURCES_DIR, SCHEMA_JSON)
motbx_collection.summarise(SUMMARY_CSV_LATEST, validate=False,
                           summary_csv_path_old=SUMMARY_CSV_PREVIOUS,
                           changelog_path=CHANGELOG_CSV)

Loading summary of previous MOTBX version  | ID0081
